In [11]:
## Importa bibliotecas necessárias
from plyer import notification
from datetime import datetime
import numpy as np
import pandas as pd
import requests
from funcoes import *
import time 
import sqlite3

### Fase de carregamento de dados:

In [12]:
## Carrega informações de estados brasileiros
df_estados_raw = carregar_dados_estados()

In [13]:
## Carrega informações de todas as cidades de todos os estados brasileiros, utilizando dataframe anterior como input para carregar as informações

df_cidades_raw = pd.DataFrame(columns = ['Nome Cidade', 'Cód. IBGE Cidade', 'Sigla Estado'])

for estado in df_estados_raw['sigla']:
    # Para cada estado da nação, ele carrega as informações em um dataframe auxiliar 'df_aux'
    df_aux = carregar_dados_cidades(estado)
    df_aux['Sigla Estado'] = estado
    df_aux.columns = ['Nome Cidade', 'Cód. IBGE Cidade', 'Sigla Estado']
    # e depois vai concatenando as informações em um único dataframe 'df_cidades_raw'
    df_cidades_raw = pd.concat([df_cidades_raw, df_aux], ignore_index= True)


In [14]:
## Carrega informações para criar uma base de dados de todos os DDDs nacionais, utilizando a mesma estratégia para criar o dataframe 'df_cidades_curated'
df_DDD_raw = pd.DataFrame(columns = ['Estado', 'Cidade', 'DDD'])

# Obs: Para evitar tentar buscar DDDs inválidos, foi pesquisado quais são os DDDs válidos atuais e são extraídas informações somente destes DDDs
DDDs_validos = ['11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '24', '27', '28', '31', '32', '33', '34', '35', '37', '38', '41', '42', '43', '44', '45', '46', '47', '48', '49', '51', '53', '54', '55', '61', '62', '63', '64', '65', '66', '67', '68', '69', '71', '73', '74', '75', '77', '79', '81', '82', '83', '84', '85', '86', '87', '88', '89', '91', '92', '93', '94', '95', '96', '97', '98', '99']
for i in DDDs_validos:
    df_aux = carregar_info_DDDs(i)
    if type(df_aux) == tuple:
        # Caso o DDD pesquisado seja inválido, continua para o próximo a ser pesquisado
        pass
    else:
        df_aux['DDD'] = i
        df_aux.columns = ['Estado', 'Cidade', 'DDD']
        df_DDD_raw = pd.concat([df_DDD_raw, df_aux], ignore_index = True)
        # Foi necessário incluir um intervalo de 1s para conseguirmos buscar a base completa de DDDs do site brasilapi
        time.sleep(1)

In [15]:
## Carrega dados adicionais retirados do site do IBGE para compor dados de todos os municípios do Brasil.
## Dados disponíveis em: https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?=&t=downloads

df_pop_raw = pd.read_excel('POP2022_Municipios_20230622.xls', sheet_name='Municípios', header=1)

### Grava base de dados raw:

In [16]:
salva_db(df_estados_raw, 'dados_raw.db', 'estados_raw')
salva_db(df_cidades_raw, 'dados_raw.db', 'cidades_raw')
salva_db(df_DDD_raw, 'dados_raw.db', 'DDDs_raw')
salva_db(df_pop_raw, 'dados_raw.db', 'populacao_raw')

### Leitura da base de dados raw:

In [17]:
df_estados = carrega_db('dados_raw.db', 'estados_raw')
df_cidades = carrega_db('dados_raw.db', 'cidades_raw')
df_DDD = carrega_db('dados_raw.db', 'DDDs_raw')
df_pop = carrega_db('dados_raw.db', 'populacao_raw')

### Fase de tratamento de dados:

In [18]:
## Planilha do IBGE possui rodapé que não contém dados da POPULAÇÃO. Estamos removendo essas linhas desnecessárias
df_pop = df_pop.dropna(subset = 'POPULAÇÃO')

In [19]:
## Padronizando Dtypes que estavam como números para strings

df_estados['id'] = df_estados['id'].astype(str)
df_pop['COD. UF'] = df_pop['COD. UF'].astype(int).astype(str)
df_pop['COD. MUNIC'] = df_pop['COD. MUNIC'].astype(int).astype(str)
df_pop['POPULAÇÃO'] = df_pop['POPULAÇÃO'].astype(str)

In [20]:
## Padronizando sempre utilizar caixa alta para textos nos dataframes
caixa_alta = lambda df_aux: df_aux.apply(lambda x: x.str.upper() if x.dtype == 'object' else x)

df_estados = caixa_alta(df_estados)
df_cidades = caixa_alta(df_cidades)
df_DDD = caixa_alta(df_DDD)
df_pop = caixa_alta(df_pop)

In [21]:
## Dados do arquivo do IBGE estavam com casos como o exemplo a seguir:
"""
37.464
461.748(1)
19.316
3.476
"""
# Queremos retirar os pontos '.' e também tratar os casos que contém parênteses, para depois transformar os dados para inteiro:
df_pop['POPULAÇÃO'] = df_pop['POPULAÇÃO'].str.replace('.', '')

# Função lambda que retorna a string até a abertura de parêntese caso houver
string_ate_parentese = lambda texto: texto.split('(')[0] if '(' in texto else texto
df_pop['POPULAÇÃO'] = list(map(string_ate_parentese, df_pop['POPULAÇÃO']))

# Transforma os dados para inteiro novamente
df_pop['POPULAÇÃO'] = df_pop['POPULAÇÃO'].astype(int)

In [22]:
## Criação de coluna Cód.IBGE Cidade debtri do dataframe que contém dados de população (necessário futuramente para conectar com dataframe de cidades)
df_pop['Cód. IBGE Cidade'] = df_pop['COD. UF'] + df_pop['COD. MUNIC'].str.zfill(5)

In [23]:
## Renomeando colunas
df_estados.columns = ['ID', 'Sigla', 'Nome', 'Região']
df_pop.columns = ['UF', 'Cód. UF', 'Cód. Município', 'Nome Município', 'População', 'Cód. IBGE Cidade']

In [29]:
df_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      27 non-null     object
 1   Sigla   27 non-null     object
 2   Nome    27 non-null     object
 3   Região  27 non-null     object
dtypes: object(4)
memory usage: 996.0+ bytes


In [30]:
df_cidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Nome Cidade       5570 non-null   object
 1   Cód. IBGE Cidade  5570 non-null   object
 2   Sigla Estado      5570 non-null   object
dtypes: object(3)
memory usage: 130.7+ KB


In [31]:
df_DDD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Estado  5565 non-null   object
 1   Cidade  5565 non-null   object
 2   DDD     5565 non-null   object
dtypes: object(3)
memory usage: 130.6+ KB


In [32]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   UF                5570 non-null   object
 1   Cód. UF           5570 non-null   object
 2   Cód. Município    5570 non-null   object
 3   Nome Município    5570 non-null   object
 4   População         5570 non-null   int32 
 5   Cód. IBGE Cidade  5570 non-null   object
dtypes: int32(1), object(5)
memory usage: 282.9+ KB


### Grava base de dados curated

In [28]:
salva_db(df_estados, 'dados_curated.db', 'estados_curated')
salva_db(df_cidades, 'dados_curated.db', 'cidades_curated')
salva_db(df_DDD, 'dados_curated.db', 'DDDs_curated')
salva_db(df_pop, 'dados_curated.db', 'populacao_curated')